In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import reward as rw
import reward.utils as U

/home/quazar/anaconda3/envs/torchrl_up/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
GAMMA = 0.99

In [3]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [4]:
env = rw.envs.GymEnv("CartPole-v0")
runner = rw.runners.SingleRunner(env)
batcher = rw.batchers.RolloutBatcher(runner=runner, batch_size=512)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
class PolicyNN(nn.Module):
    def __init__(self, num_inputs, num_outputs, activation=nn.Tanh):
        super().__init__()
        self.activation = activation()
        
        self.hidden = nn.Linear(num_inputs, 64)
        self.out = nn.Linear(64, num_outputs)
        
    def forward(self, x):
        x = self.activation(self.hidden(x))
        return self.out(x)  

In [6]:
class CategoricalPolicy(rw.policy.BasePolicy):
    def create_dist(self, state):
        logits = self.nn(state)
        return rw.distributions.Categorical(logits=logits)
    
    def get_action(self, state, step):
        dist = self.create_dist(state)
        return U.to_np(dist.sample())

In [7]:
p_nn = PolicyNN(batcher.state_info.shape[0], batcher.action_info.shape).to(device)
policy = CategoricalPolicy(p_nn)

p_opt = torch.optim.Adam(p_nn.parameters(), lr=1e-3)

In [8]:
logger = U.Logger('logs/cart_pole/pg-v0-3')
last_logged_step = 0

Writing logs to: logs/pg-v0-3


In [9]:
for batch in batcher.get_batches(max_steps=3e5, get_action_fn=policy.get_action):
    batch = batch.to_tensor()
    # TODO: Calculation is actually a little wrong here, v_t_last should not be None
    ret = U.estimators.discounted_sum_rewards(rewards=batch.reward, dones=batch.done, gamma=GAMMA)
    batch = batch.concat_batch()
    
    # Calculate loss
    dist = policy.create_dist(batch.state_t)
    log_prob = dist.log_prob(batch.action)
    assert ret.shape == log_prob.shape
    losses = ret * log_prob
    loss = -losses.mean()
    
    # Optimize
    p_opt.zero_grad()
    loss.backward()
    p_opt.step()
    
    # Write logs    
    if batcher.num_steps > last_logged_step:
        last_logged_step = batcher.num_steps + 10000
        batcher.write_logs(logger)
        logger.add_log('policy/loss', loss)
        logger.log(step=batcher.num_steps)    


--------------------------------------------------------------
Env/Reward/Episode (New)                               | 20.71
Env/Length/Episode (New)                               | 20.71
Env/Reward/Episode (Last 50)                           | 20.71
Env/Length/Episode (Last 50)                           | 20.71
policy/loss                                            |  7.96
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                               | 25.67
Env/Length/Episode (New)                               | 25.67
Env/Reward/Episode (Last 50)                           | 32.40
Env/Length/Episode (Last 50)                           | 32.40
policy/loss                                            | 10.95
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            


--------------------------------------------------------------
Env/Reward/Episode (New)                              | 194.21
Env/Length/Episode (New)                              | 194.21
Env/Reward/Episode (Last 50)                          | 193.86
Env/Length/Episode (Last 50)                          | 193.86
policy/loss                                           |  28.43
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                              | 195.17
Env/Length/Episode (New)                              | 195.17
Env/Reward/Episode (Last 50)                          | 194.98
Env/Length/Episode (Last 50)                          | 194.98
policy/loss                                           |  27.15
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            